 # Using met.no forecasts with weather-tools



 This notebook-style script shows how to fetch met.no forecasts and combine them

 with SILO observations using the new `weather_tools` APIs. Open it directly in

 JupyterLab or VS Code with Jupytext to run the cells interactively.

 ## Prerequisites

 - Network access to `https://api.met.no` and a valid contact e-mail for the User-Agent header.

 - Set `SILO_API_KEY` (your registered SILO e-mail) or pass `api_key=` directly to `SiloAPI`.

 - Install `weather-tools` in your environment (either via `pip install weather-tools` or the local checkout).

 - Optional: enable logging (e.g. `logging.basicConfig(level=logging.INFO)`) to inspect requests.

In [1]:
import pandas as pd

from weather_tools.merge_weather_data import get_merge_summary, merge_historical_and_forecast
from weather_tools.metno_api import MetNoAPI
from weather_tools.metno_models import MetNoFormat, MetNoQuery
from weather_tools.silo_api import SiloAPI
from weather_tools.silo_models import AustralianCoordinates


 ## 1. Configure the met.no API client

 met.no requires a descriptive User-Agent string that includes contact details.

 Replace the placeholder below with your own application name and e-mail.

In [2]:
api = MetNoAPI(
    # user_agent="weather-tools-example/0.1 (contact: you@example.com)",
    enable_cache=True,
    log_level="DEBUG",
)


 ## 2. Build a forecast query for your location

 The `AustralianCoordinates` model validates that the latitude and longitude sit

 within the range supported by SILO (GDA94 datum). Adjust the coordinates to

 target your site of interest.

In [3]:
coordinates = AustralianCoordinates(latitude=-33.86, longitude=151.21)  # Sydney, NSW
query = MetNoQuery(coordinates=coordinates, format=MetNoFormat.COMPACT)

response = api.query_forecast(query)
metadata = response.get_meta()

metadata


[10/23/25 18:34:02] DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Making request (attempt 1/3):                                                         
                             https://api.met.no/weatherapi/locationforecast/2.0/compact

                    DEBUG    Starting new HTTPS connection (1): api.met.no:443

[10/23/25 18:34:03] DEBUG    https://api.met.no:443 "GET                                                           
                             /weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21 HTTP/1.1" 200 2905

                    DEBUG    Cached response for: 4a5961d27789c123038238e4def09a11

                    INFO     Request successful on attempt 1

{'updated_at': '2025-10-23T09:16:44Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'cloud_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

In [4]:
response

MetNoResponse(raw_data={'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [151.21, -33.86, 24]}, 'properties': {'meta': {'updated_at': '2025-10-23T09:16:44Z', 'units': {'air_pressure_at_sea_level': 'hPa', 'air_temperature': 'celsius', 'cloud_area_fraction': '%', 'precipitation_amount': 'mm', 'relative_humidity': '%', 'wind_from_direction': 'degrees', 'wind_speed': 'm/s'}}, 'timeseries': [{'time': '2025-10-23T10:00:00Z', 'data': {'instant': {'details': {'air_pressure_at_sea_level': 1014.8, 'air_temperature': 18.4, 'cloud_area_fraction': 86.7, 'relative_humidity': 61.7, 'wind_from_direction': 65.1, 'wind_speed': 6.3}}, 'next_12_hours': {'summary': {'symbol_code': 'partlycloudy_night'}, 'details': {}}, 'next_1_hours': {'summary': {'symbol_code': 'cloudy'}, 'details': {'precipitation_amount': 0.0}}, 'next_6_hours': {'summary': {'symbol_code': 'partlycloudy_night'}, 'details': {'precipitation_amount': 0.0}}}}, {'time': '2025-10-23T11:00:00Z', 'data': {'instant': {'details': {'

 ## 3. Convert forecasts to daily and hourly tables

 `MetNoAPI.to_dataframe` can aggregate the hourly GeoJSON payload into the daily

 schema (`DailyWeatherSummary`) or expose the raw hourly data by disabling the

 aggregation flag.

In [5]:
daily_forecasts = api.to_dataframe(response, aggregate_to_daily=True)
hourly_forecasts = api.to_dataframe(response, aggregate_to_daily=False)

daily_forecasts.head()


,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-23,16.1,21.2,NaN,4.121429,6.3,65.635714,1013.728571,45.135714,cloudy
1,2025-10-24,15.1,23.9,0.1,5.554167,11.1,71.045833,1015.545833,69.983333,lightrain
2,2025-10-25,15.8,22.1,NaN,5.662500,9.9,73.633333,1011.554167,63.379167,partlycloudy_day
3,2025-10-26,20.1,28.0,0.5,3.325000,4.7,48.425000,1000.850000,33.800000,cloudy
4,2025-10-27,14.6,19.7,1.7,8.125000,11.9,56.550000,1013.300000,91.575000,rainshowers_day


In [6]:
hourly_forecasts.head()


,time,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,next_1_hours_precipitation_amount,next_6_hours_precipitation_amount
0,2025-10-23 10:00:00+00:00,1014.8,18.4,86.7,61.7,65.1,6.3,0.0,NaN
1,2025-10-23 11:00:00+00:00,1015.4,18.4,89.8,62.3,43.0,5.5,0.0,NaN
2,2025-10-23 12:00:00+00:00,1014.9,18.4,56.2,61.8,34.5,5.5,0.0,NaN
3,2025-10-23 13:00:00+00:00,1015.0,18.4,99.2,62.4,17.5,4.2,0.0,NaN
4,2025-10-23 14:00:00+00:00,1014.1,18.3,33.6,64.3,10.0,3.8,0.0,NaN


 ### Quick helper: `get_daily_forecast`

 Prefer `get_daily_forecast` if you simply need the daily summaries as Pydantic

 models and want to control the number of days returned.

In [7]:
summaries = api.get_daily_forecast(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    days=5,
)

pd.DataFrame([summary.model_dump() for summary in summaries])


                    DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Cache hit for request: 4a5961d27789c123038238e4def09a11

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-23,16.1,21.2,NaN,4.121429,6.3,65.635714,1013.728571,45.135714,cloudy
1,2025-10-24,15.1,23.9,0.1,5.554167,11.1,71.045833,1015.545833,69.983333,lightrain
2,2025-10-25,15.8,22.1,NaN,5.662500,9.9,73.633333,1011.554167,63.379167,partlycloudy_day
3,2025-10-26,20.1,28.0,0.5,3.325000,4.7,48.425000,1000.850000,33.800000,cloudy
4,2025-10-27,14.6,19.7,1.7,8.125000,11.9,56.550000,1013.300000,91.575000,rainshowers_day


 ## 4. Merge met.no forecasts with SILO history

 Pull the last five days from the SILO DataDrill API for the same coordinates,

 then merge that history with the met.no forecast. The helper automatically

 converts column names, fills optional variables (when enabled), and annotates

 the data source for each record.

In [8]:
first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date - pd.Timedelta(days=1)
history_start = history_end - pd.Timedelta(days=4)

silo_api = SiloAPI()
raw_silo = silo_api.get_gridded_data(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    start_date=history_start.strftime("%Y%m%d"),
    end_date=history_end.strftime("%Y%m%d"),
    variables=["rainfall", "max_temp", "min_temp"],
)

silo_history = raw_silo.rename(columns={"YYYY-MM-DD": "date"}).copy()

silo_history["date"] = pd.to_datetime(silo_history["date"])

required_cols = ["date", "min_temp", "max_temp", "daily_rain"]
missing = [col for col in required_cols if col not in silo_history.columns]
if missing:
    raise ValueError(f"Missing required columns from SILO response: {missing}")

silo_data = silo_history[required_cols].sort_values("date").reset_index(drop=True)

merged = merge_historical_and_forecast(
    silo_data=silo_data,
    metno_data=daily_forecasts,
    fill_missing=True,
    overlap_strategy="prefer_silo",
)

merged


                    DEBUG    🌐 Constructed URL:                                                                   
                             https://www.longpaddock.qld.gov.au/cgi-bin/silo/DataDrillDataset.php?lat=-33.86&lon=15
                             1.21&start=20251018&finish=20251022&format=csv&username=harry.eslick@dpird.wa.gov.au&p
                             assword=apirequest&comment=RXN

                    DEBUG    Making request (attempt 1/3):                                                         
                             https://www.longpaddock.qld.gov.au/cgi-bin/silo/DataDrillDataset.php

                    DEBUG    Starting new HTTPS connection (1): www.longpaddock.qld.gov.au:443

[10/23/25 18:34:04] DEBUG    https://www.longpaddock.qld.gov.au:443 "GET                                           
                             /cgi-bin/silo/DataDrillDataset.php?lat=-33.86&lon=151.21&start=20251018&finish=2025102
                             2&format=csv&username=harry.eslick%40dpird.wa.gov.au&password=apirequest&comment=RXN  
                             HTTP/1.1" 200 593

                    INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-23 00:00:00

/Users/hfsi/Developer/weather_tools/src/weather_tools/merge_weather_data.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([silo_df, metno_df], ignore_index=True)


                    INFO     Merged 5 SILO records with 10 met.no records. Total: 15 records

,date,is_forecast,max_wind_speed,dominant_weather_symbol,data_source,forecast_generated_at,min_temp,mslp,avg_wind_speed,avg_cloud_fraction,daily_rain,max_temp,vp
0,2025-10-18,False,NaN,NaN,silo,NaT,15.7,NaN,NaN,NaN,0.7,22.0,NaN
1,2025-10-19,False,NaN,NaN,silo,NaT,15.5,NaN,NaN,NaN,0.0,25.5,NaN
2,2025-10-20,False,NaN,NaN,silo,NaT,17.0,NaN,NaN,NaN,0.0,36.0,NaN
3,2025-10-21,False,NaN,NaN,silo,NaT,17.7,NaN,NaN,NaN,0.0,23.7,NaN
4,2025-10-22,False,NaN,NaN,silo,NaT,18.7,NaN,NaN,NaN,0.0,22.8,NaN
5,2025-10-23,True,6.3,cloudy,metno,2025-10-23 10:34:04.041614+00:00,16.1,1013.728571,4.121429,45.135714,NaN,21.2,65.635714
6,2025-10-24,True,11.1,lightrain,metno,2025-10-23 10:34:04.041614+00:00,15.1,1015.545833,5.554167,69.983333,0.1,23.9,71.045833
7,2025-10-25,True,9.9,partlycloudy_day,metno,2025-10-23 10:34:04.041614+00:00,15.8,1011.554167,5.662500,63.379167,NaN,22.1,73.633333
8,2025-10-26,True,4.7,cloudy,metno,2025-10-23 10:34:04.041614+00:00,20.1,1000.850000,3.325000,33.800000,0.5,28.0,48.425000
9,2025-10-27,True,11.9,rainshowers_day,metno,2025-10-23 10:34:04.041614+00:00,14.6,1013.300000,8.125000,91.575000,1.7,19.7,56.550000


In [9]:
merge_summary = get_merge_summary(merged)
merge_summary


{'total_records': 15,
 'silo_records': np.int64(5),
 'metno_records': np.int64(10),
 'date_range': {'start': Timestamp('2025-10-18 00:00:00'),
  'end': Timestamp('2025-11-01 00:00:00'),
  'days': 15},
 'silo_period': {'start': Timestamp('2025-10-18 00:00:00'),
  'end': Timestamp('2025-10-22 00:00:00')},
 'metno_period': {'start': Timestamp('2025-10-23 00:00:00'),
  'end': Timestamp('2025-11-01 00:00:00')},
 'transition_date': Timestamp('2025-10-23 00:00:00')}

In [10]:
## 4b. Merge Forecast with SILO PatchedPoint Data

first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date #- pd.Timedelta(days=1) # Include day before forecast to check overlap handling
history_start = history_end - pd.Timedelta(days=8)

silo_api = SiloAPI(log_level="INFO")
raw_silo = silo_api.search_stations("Northam")

# {'station_code': 10111,
#  'name': 'NORTHAM',
#  'latitude': -31.651,
#  'longitude': 116.659,
#  'state': 'WA',
#  'elevation': 170.0}

silo_station_data = silo_api.get_station_data(station_code="10111",
						  start_date=history_start.strftime("%Y%m%d"),
						end_date=history_end.strftime("%Y%m%d"),
						variables=["rainfall", "max_temp", "min_temp", "evaporation"],
		)

silo_station_data = silo_station_data.rename(columns={"YYYY-MM-DD": "date"}).copy()


merge_historical_and_forecast(
    silo_data=silo_station_data,
    metno_data=daily_forecasts,
    fill_missing=True,
    overlap_strategy="prefer_silo",
)



[10/23/25 18:34:05] DEBUG    🌐 Constructed URL:                                                                   
                             https://www.longpaddock.qld.gov.au/cgi-bin/silo/PatchedPointDataset.php?format=name&na
                             meFrag=Northam

                    DEBUG    Making request (attempt 1/3):                                                         
                             https://www.longpaddock.qld.gov.au/cgi-bin/silo/PatchedPointDataset.php

                    DEBUG    Starting new HTTPS connection (1): www.longpaddock.qld.gov.au:443

                    DEBUG    https://www.longpaddock.qld.gov.au:443 "GET                                           
                             /cgi-bin/silo/PatchedPointDataset.php?format=name&nameFrag=Northam HTTP/1.1" 200 240

                    INFO     Request successful on attempt 1

...

                    INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-23 00:00:00

/Users/hfsi/Developer/weather_tools/src/weather_tools/merge_weather_data.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([silo_df, metno_df], ignore_index=True)


                    INFO     Merged 8 SILO records with 10 met.no records. Total: 18 records

,date,station,is_forecast,evap_pan,daily_rain_source,max_temp_source,max_wind_speed,dominant_weather_symbol,min_temp_source,evap_pan_source,data_source,forecast_generated_at,min_temp,mslp,avg_wind_speed,avg_cloud_fraction,daily_rain,max_temp,vp
0,2025-10-15,10111.0,False,4.6,0.0,0.0,NaN,NaN,0.0,25.0,silo,NaT,10.6,NaN,NaN,NaN,0.0,24.7,NaN
1,2025-10-16,10111.0,False,6.7,0.0,0.0,NaN,NaN,0.0,25.0,silo,NaT,8.0,NaN,NaN,NaN,0.0,27.7,NaN
2,2025-10-17,10111.0,False,6.8,0.0,0.0,NaN,NaN,0.0,25.0,silo,NaT,9.1,NaN,NaN,NaN,0.0,34.0,NaN
3,2025-10-18,10111.0,False,6.9,0.0,0.0,NaN,NaN,0.0,25.0,silo,NaT,15.9,NaN,NaN,NaN,0.0,24.4,NaN
4,2025-10-19,10111.0,False,4.9,0.0,0.0,NaN,NaN,0.0,25.0,silo,NaT,7.5,NaN,NaN,NaN,0.0,22.9,NaN
5,2025-10-20,10111.0,False,3.2,0.0,0.0,NaN,NaN,0.0,25.0,silo,NaT,6.0,NaN,NaN,NaN,0.0,21.4,NaN
6,2025-10-21,10111.0,False,4.5,0.0,0.0,NaN,NaN,0.0,25.0,silo,NaT,9.0,NaN,NaN,NaN,19.8,22.5,NaN
7,2025-10-22,10111.0,False,0.0,0.0,75.0,NaN,NaN,0.0,0.0,silo,NaT,8.7,NaN,NaN,NaN,0.0,25.5,NaN
8,2025-10-23,NaN,True,NaN,NaN,NaN,6.3,cloudy,NaN,NaN,metno,2025-10-23 10:34:05.869145+00:00,16.1,1013.728571,4.121429,45.135714,NaN,21.2,65.635714
9,2025-10-24,NaN,True,NaN,NaN,NaN,11.1,lightrain,NaN,NaN,metno,2025-10-23 10:34:05.869145+00:00,15.1,1015.545833,5.554167,69.983333,0.1,23.9,71.045833


In [48]:
daily_forecasts.columns

Index(['date', 'min_temperature', 'max_temperature', 'total_precipitation',
       'avg_wind_speed', 'max_wind_speed', 'avg_relative_humidity',
       'avg_pressure', 'avg_cloud_fraction', 'dominant_weather_symbol'],
      dtype='object')